
## Overview

This notebook will show you how to create and query a table or DataFrame that you uploaded to DBFS. [DBFS](https://docs.databricks.com/user-guide/dbfs-databricks-file-system.html) is a Databricks File System that allows you to store data for querying inside of Databricks. This notebook assumes that you have a file already inside of DBFS that you would like to read from.

This notebook is written in **Python** so the default cell type is Python. However, you can use different languages by using the `%LANGUAGE` syntax. Python, Scala, SQL, and R are all supported.

In [0]:
# File location and type
f1_location = "/FileStore/tables/members.csv"
f2_location = "/FileStore/tables/bookings.csv"
f3_location = "/FileStore/tables/facilities.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
members_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(f1_location)

bookings_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(f2_location)

facilities_df = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(f3_location)

In [0]:
# Create a view or table

temp_bookings = "bookings"
temp_members = "members"
temp_facilities = "facilities"

members_df.createOrReplaceTempView(temp_members)
bookings_df.createOrReplaceTempView(temp_bookings)
facilities_df.createOrReplaceTempView(temp_facilities)

In [0]:
# -- select * from facilities limit 5
facilities_df.limit(5).show()

+-----+---------------+----------+---------+-------------+------------------+
|facid|           name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+---------------+----------+---------+-------------+------------------+
|    0| Tennis Court 1|       5.0|     25.0|        10000|               200|
|    1| Tennis Court 2|       5.0|     25.0|         8000|               200|
|    2|Badminton Court|       0.0|     15.5|         4000|                50|
|    3|   Table Tennis|       0.0|      5.0|          320|                10|
|    4| Massage Room 1|      35.0|     80.0|         4000|              3000|
+-----+---------------+----------+---------+-------------+------------------+




## Join Exercises

In [0]:
# %sql
# SELECT b.starttime
# FROM members m
# JOIN bookings b ON m.memid = b.memid
# WHERE m.firstname LIKE 'David'AND m.surname LIKE 'Farrell'

bookings_df.join(members_df, bookings_df.memid == members_df.memid)\
    .filter((members_df.firstname == 'David') & (members_df.surname == 'Farrell')) \
    .select(bookings_df.starttime) \
    .show()

+-------------------+
|          starttime|
+-------------------+
|2012-09-18 09:00:00|
|2012-09-18 17:30:00|
|2012-09-18 13:30:00|
|2012-09-18 20:00:00|
|2012-09-19 09:30:00|
|2012-09-19 15:00:00|
|2012-09-19 12:00:00|
|2012-09-20 15:30:00|
|2012-09-20 11:30:00|
|2012-09-20 14:00:00|
|2012-09-21 10:30:00|
|2012-09-21 14:00:00|
|2012-09-22 08:30:00|
|2012-09-22 17:00:00|
|2012-09-23 08:30:00|
|2012-09-23 17:30:00|
|2012-09-23 19:00:00|
|2012-09-24 08:00:00|
|2012-09-24 16:30:00|
|2012-09-24 12:30:00|
+-------------------+
only showing top 20 rows



In [0]:
# %sql
# SELECT b.starttime, f.name
# FROM facilities f
# INNER JOIN bookings b ON f.facid = b.facid
# WHERE f.name LIKE 'Tennis Court%'AND b.starttime BETWEEN '2012-09-21' AND '2012-09-22'
# ORDER BY b.starttime;

bookings_df.join(facilities_df, bookings_df.facid == facilities_df.facid) \
    .filter((facilities_df.name.like('Tennis Court%')) & (bookings_df.starttime.between('2012-09-21', '2012-09-22'))) \
    .select(bookings_df.starttime, facilities_df.name) \
    .orderBy(bookings_df.starttime) \
    .show()


+-------------------+--------------+
|          starttime|          name|
+-------------------+--------------+
|2012-09-21 08:00:00|Tennis Court 1|
|2012-09-21 08:00:00|Tennis Court 2|
|2012-09-21 09:30:00|Tennis Court 1|
|2012-09-21 10:00:00|Tennis Court 2|
|2012-09-21 11:30:00|Tennis Court 2|
|2012-09-21 12:00:00|Tennis Court 1|
|2012-09-21 13:30:00|Tennis Court 1|
|2012-09-21 14:00:00|Tennis Court 2|
|2012-09-21 15:30:00|Tennis Court 1|
|2012-09-21 16:00:00|Tennis Court 2|
|2012-09-21 17:00:00|Tennis Court 1|
|2012-09-21 18:00:00|Tennis Court 2|
+-------------------+--------------+



In [0]:
# %sql
# SELECT 
# m.firstname AS mfname, m.surname AS msname, r.firstname AS rfname, r.surname AS rsname
# FROM members m
# LEFT OUTER JOIN members r ON r.memid = m.recommendedby
# ORDER BY msname, mfname;

m = members_df.alias("m")
r = members_df.alias("r")

m.join(r, col("r.memid") == col("m.recommendedby"), "left") \
    .select(
        col("m.firstname").alias("mfname"),
        col("m.surname").alias("msname"),
        col("r.firstname").alias("rfname"),
        col("r.surname").alias("rsname")
    ) \
    .orderBy("msname", "mfname") \
    .show(n=100, truncate=False)

+---------+-----------------+---------+--------+
|mfname   |msname           |rfname   |rsname  |
+---------+-----------------+---------+--------+
|Florence |Bader            |Ponder   |Stibbons|
|Anne     |Baker            |Ponder   |Stibbons|
|Timothy  |Baker            |Jemima   |Farrell |
|Tim      |Boothe           |Tim      |Rownam  |
|Gerald   |Butters          |Darren   |Smith   |
|Joan     |Coplin           |Timothy  |Baker   |
|Erica    |Crumpet          |Tracy    |Smith   |
|Nancy    |Dare             |Janice   |Joplette|
|David    |Farrell          |null     |null    |
|Jemima   |Farrell          |null     |null    |
|GUEST    |GUEST            |null     |null    |
|Matthew  |Genting          |Gerald   |Butters |
|John     |Hunt             |Millicent|Purview |
|David    |Jones            |Janice   |Joplette|
|Douglas  |Jones            |David    |Jones   |
|Janice   |Joplette         |Darren   |Smith   |
|Anna     |Mackenzie        |Darren   |Smith   |
|Charles  |Owen     

In [0]:
# %sql
# SELECT DISTINCT m.firstname || ' ' || m.surname AS member, f.name as facility
# FROM members m
# JOIN bookings b ON m.memid = b.memid
# JOIN facilities f ON b.facid = f.facid
# WHERE f.name LIKE 'Tennis Court%'
# ORDER BY member, facility;
from pyspark.sql.functions import concat_ws

members_df.join(bookings_df, members_df.memid == bookings_df.memid) \
    .join(facilities_df, bookings_df.facid == facilities_df.facid) \
    .filter(facilities_df.name.like("Tennis Court%")) \
    .select(
        concat_ws(" ", members_df.firstname, members_df.surname).alias("member"),
        facilities_df.name.alias("facility")
    ) \
    .distinct() \
    .orderBy("member", "facility") \
    .show(n=100, truncate=False)

+-----------------+--------------+
|member           |facility      |
+-----------------+--------------+
|Anne Baker       |Tennis Court 1|
|Anne Baker       |Tennis Court 2|
|Burton Tracy     |Tennis Court 1|
|Burton Tracy     |Tennis Court 2|
|Charles Owen     |Tennis Court 1|
|Charles Owen     |Tennis Court 2|
|Darren Smith     |Tennis Court 2|
|David Farrell    |Tennis Court 1|
|David Farrell    |Tennis Court 2|
|David Jones      |Tennis Court 1|
|David Jones      |Tennis Court 2|
|David Pinker     |Tennis Court 1|
|Douglas Jones    |Tennis Court 1|
|Erica Crumpet    |Tennis Court 1|
|Florence Bader   |Tennis Court 1|
|Florence Bader   |Tennis Court 2|
|GUEST GUEST      |Tennis Court 1|
|GUEST GUEST      |Tennis Court 2|
|Gerald Butters   |Tennis Court 1|
|Gerald Butters   |Tennis Court 2|
|Henrietta Rumney |Tennis Court 2|
|Jack Smith       |Tennis Court 1|
|Jack Smith       |Tennis Court 2|
|Janice Joplette  |Tennis Court 1|
|Janice Joplette  |Tennis Court 2|
|Jemima Farrell   |T

In [0]:
# %sql
# SELECT DISTINCT 
#     m.firstname || ' ' || m.surname AS member,
#     r.firstname || ' ' || r.surname AS recommender
# FROM members m
# LEFT JOIN members r ON r.memid = m.recommendedby
# ORDER BY member;
m = members_df.alias("m")
r = members_df.alias("r")

m.join(r, col("r.memid") == col("m.recommendedby"), "left") \
    .select(
        concat_ws(" ", col("m.firstname"), col("m.surname")).alias("member"),
        concat_ws(" ", col("r.firstname"), col("r.surname")).alias("recommender")
    ) \
    .distinct() \
    .orderBy("member") \
    .show(n=1000, truncate=False)


+-----------------------+-----------------+
|member                 |recommender      |
+-----------------------+-----------------+
|Anna Mackenzie         |Darren Smith     |
|Anne Baker             |Ponder Stibbons  |
|Burton Tracy           |                 |
|Charles Owen           |Darren Smith     |
|Darren Smith           |                 |
|David Farrell          |                 |
|David Jones            |Janice Joplette  |
|David Pinker           |Jemima Farrell   |
|Douglas Jones          |David Jones      |
|Erica Crumpet          |Tracy Smith      |
|Florence Bader         |Ponder Stibbons  |
|GUEST GUEST            |                 |
|Gerald Butters         |Darren Smith     |
|Henrietta Rumney       |Matthew Genting  |
|Henry Worthington-Smyth|Tracy Smith      |
|Hyacinth Tupperware    |                 |
|Jack Smith             |Darren Smith     |
|Janice Joplette        |Darren Smith     |
|Jemima Farrell         |                 |
|Joan Coplin            |Timothy


## Aggregation

In [0]:
# %sql
# SELECT m.recommendedby, COUNT(*)
# FROM members as m
# WHERE recommendedby is not null
# GROUP BY recommendedby
# ORDER BY recommendedby;
from pyspark.sql import functions as F

m = members_df.alias("m")

m.filter(F.col("m.recommendedby").isNotNull()) \
    .groupBy("m.recommendedby") \
    .agg(F.count("*").alias("count")) \
    .orderBy("m.recommendedby") \
    .show()



+-------------+-----+
|recommendedby|count|
+-------------+-----+
|            1|    5|
|            2|    3|
|            3|    1|
|            4|    2|
|            5|    1|
|            6|    1|
|            9|    2|
|           11|    1|
|           13|    2|
|           15|    1|
|           16|    1|
|           20|    1|
|           30|    1|
+-------------+-----+



In [0]:
# %sql
# SELECT b.facid, SUM(slots) AS total_slots
# FROM bookings as b
# GROUP BY b.facid
# ORDER BY b.facid;

b = bookings_df.alias("b")

# Correct the filter and ensure proper syntax
b.filter(F.col("b.facid").isNotNull()) \
  .groupBy("b.facid") \
  .agg(F.sum("b.slots").alias("Total Slots")) \
  .orderBy("b.facid") \
  .show()

+-----+-----------+
|facid|Total Slots|
+-----+-----------+
|    0|       1320|
|    1|       1278|
|    2|       1209|
|    3|        830|
|    4|       1404|
|    5|        228|
|    6|       1104|
|    7|        908|
|    8|        911|
+-----+-----------+



In [0]:
# %sql
# SELECT b.facid, SUM(b.slots) AS total_slots
# FROM bookings AS b
# WHERE b.starttime >= '2012-09-01' AND b.starttime < '2012-10-01'
# GROUP BY b.facid
# ORDER BY total_slots ASC;

b = bookings_df.alias("b")

b.filter(F.col("b.facid").isNotNull() & 
         F.col("b.starttime").between('2012-09-01', '2012-10-01')) \
  .groupBy("b.facid") \
  .agg(F.sum("b.slots").alias("Total_Slots")) \
  .orderBy("Total_Slots") \
  .show()

+-----+-----------+
|facid|Total_Slots|
+-----+-----------+
|    5|        122|
|    3|        422|
|    7|        426|
|    8|        471|
|    6|        540|
|    2|        570|
|    1|        588|
|    0|        591|
|    4|        648|
+-----+-----------+



In [0]:
# %sql
# SELECT b.facid, 
# EXTRACT(MONTH FROM b.starttime) AS month, 
# SUM(b.slots) AS total_slots
# FROM bookings AS b
# WHERE EXTRACT(year from starttime) = 2012
# GROUP BY facid, month
# ORDER BY facid, month;

b = bookings_df.alias("b")

b.filter(F.year("b.starttime") == 2012) \
  .groupBy("b.facid", F.month("b.starttime").alias("month")) \
  .agg(F.sum("b.slots").alias("total_slots")) \
  .orderBy("b.facid", "month") \
  .show(n=27, truncate=False)

+-----+-----+-----------+
|facid|month|total_slots|
+-----+-----+-----------+
|0    |7    |270        |
|0    |8    |459        |
|0    |9    |591        |
|1    |7    |207        |
|1    |8    |483        |
|1    |9    |588        |
|2    |7    |180        |
|2    |8    |459        |
|2    |9    |570        |
|3    |7    |104        |
|3    |8    |304        |
|3    |9    |422        |
|4    |7    |264        |
|4    |8    |492        |
|4    |9    |648        |
|5    |7    |24         |
|5    |8    |82         |
|5    |9    |122        |
|6    |7    |164        |
|6    |8    |400        |
|6    |9    |540        |
|7    |7    |156        |
|7    |8    |326        |
|7    |9    |426        |
|8    |7    |117        |
|8    |8    |322        |
|8    |9    |471        |
+-----+-----+-----------+



In [0]:
# %sql
# SELECT COUNT(DISTINCT b.memid) AS count
# FROM bookings AS b
b = bookings_df.alias("b")
b.select(F.countDistinct("b.memid").alias("count")).show()

+-----+
|count|
+-----+
|   30|
+-----+



In [0]:
# %sql
# SELECT m.surname, m.firstname, m.memid, MIN(b.starttime) AS starttime
# FROM bookings b
# INNER JOIN members m on m.memid = b.memid
# WHERE starttime >= '2012-09-01'
# GROUP BY m.surname, m.firstname, m.memid
# ORDER BY m.memid; 

b = bookings_df.alias("b")
m = members_df.alias("m")

b.join(m, b.memid == m.memid) \
  .filter(b.starttime >= '2012-09-01') \
  .groupBy(m.surname, m.firstname, m.memid) \
  .agg(F.min(b.starttime).alias("starttime")) \
  .orderBy(m.memid) \
  .select(m.surname, m.firstname, m.memid, "starttime") \
  .show(n=30, truncate=False)

+-----------------+---------+-----+-------------------+
|surname          |firstname|memid|starttime          |
+-----------------+---------+-----+-------------------+
|GUEST            |GUEST    |0    |2012-09-01 08:00:00|
|Smith            |Darren   |1    |2012-09-01 09:00:00|
|Smith            |Tracy    |2    |2012-09-01 11:30:00|
|Rownam           |Tim      |3    |2012-09-01 16:00:00|
|Joplette         |Janice   |4    |2012-09-01 15:00:00|
|Butters          |Gerald   |5    |2012-09-02 12:30:00|
|Tracy            |Burton   |6    |2012-09-01 15:00:00|
|Dare             |Nancy    |7    |2012-09-01 12:30:00|
|Boothe           |Tim      |8    |2012-09-01 08:30:00|
|Stibbons         |Ponder   |9    |2012-09-01 11:00:00|
|Owen             |Charles  |10   |2012-09-01 11:00:00|
|Jones            |David    |11   |2012-09-01 09:30:00|
|Baker            |Anne     |12   |2012-09-01 14:30:00|
|Farrell          |Jemima   |13   |2012-09-01 09:30:00|
|Smith            |Jack     |14   |2012-09-01 11


## String & Date

In [0]:
# %sql
# SELECT m.surname || ', ' || m.firstname AS name
# FROM members AS m

m = members_df.alias("m")
m.select(F.concat(F.col("m.surname"), F.lit(", "), F.col("m.firstname")).alias("name")).show(n=100, truncate=False)

+------------------------+
|name                    |
+------------------------+
|GUEST, GUEST            |
|Smith, Darren           |
|Smith, Tracy            |
|Rownam, Tim             |
|Joplette, Janice        |
|Butters, Gerald         |
|Tracy, Burton           |
|Dare, Nancy             |
|Boothe, Tim             |
|Stibbons, Ponder        |
|Owen, Charles           |
|Jones, David            |
|Baker, Anne             |
|Farrell, Jemima         |
|Smith, Jack             |
|Bader, Florence         |
|Baker, Timothy          |
|Pinker, David           |
|Genting, Matthew        |
|Mackenzie, Anna         |
|Coplin, Joan            |
|Sarwin, Ramnaresh       |
|Jones, Douglas          |
|Rumney, Henrietta       |
|Farrell, David          |
|Worthington-Smyth, Henry|
|Purview, Millicent      |
|Tupperware, Hyacinth    |
|Hunt, John              |
|Crumpet, Erica          |
|Smith, Darren           |
+------------------------+



In [0]:
# %sql
# SELECT * 
# FROM facilities 
# WHERE upper(name) LIKE 'TENNIS%';
f = facilities_df.alias("f")  
f.filter(F.upper(F.col("f.name")).like("TENNIS%")) \
  .show()

+-----+--------------+----------+---------+-------------+------------------+
|facid|          name|membercost|guestcost|initialoutlay|monthlymaintenance|
+-----+--------------+----------+---------+-------------+------------------+
|    0|Tennis Court 1|       5.0|     25.0|        10000|               200|
|    1|Tennis Court 2|       5.0|     25.0|         8000|               200|
+-----+--------------+----------+---------+-------------+------------------+



In [0]:
# %sql
# SELECT memid, telephone 
# FROM members 
# WHERE telephone RLIKE '\\(.*\\)';

m = members_df.alias("m")
m.filter(F.col("m.telephone").rlike(r"^\(\d{3}\)\s\d{3}-\d{4}$")) \
  .select("m.memid", "m.telephone") \
  .show(n=25, truncate=False)

+-----+--------------+
|memid|telephone     |
+-----+--------------+
|0    |(000) 000-0000|
|3    |(844) 693-0723|
|4    |(833) 942-4710|
|5    |(844) 078-4130|
|6    |(822) 354-9973|
|7    |(833) 776-4001|
|8    |(811) 433-2547|
|9    |(833) 160-3900|
|10   |(855) 542-5251|
|11   |(844) 536-8036|
|13   |(855) 016-0163|
|14   |(822) 163-3254|
|15   |(833) 499-3527|
|20   |(811) 972-1377|
|21   |(822) 661-2898|
|22   |(822) 499-2232|
|24   |(822) 413-1470|
|27   |(822) 989-8876|
|28   |(855) 755-9876|
|29   |(855) 894-3758|
|30   |(855) 941-9786|
|33   |(822) 665-5327|
|35   |(899) 720-6978|
|36   |(811) 732-4816|
|37   |(822) 577-3541|
+-----+--------------+



In [0]:
# %sql
# SELECT substr (m.surname,1,1) as letter, count(*) as count 
# FROM members m
# GROUP BY letter
# ORDER BY letter  
m = members_df.alias("m")

m.withColumn("letter", F.substring(F.col("m.surname"), 1, 1)) \
  .groupBy("letter") \
  .agg(F.count("*").alias("count")) \
  .orderBy("letter") \
  .show(n=26, truncate=False)


+------+-----+
|letter|count|
+------+-----+
|B     |5    |
|C     |2    |
|D     |1    |
|F     |2    |
|G     |2    |
|H     |1    |
|J     |3    |
|M     |1    |
|O     |1    |
|P     |2    |
|R     |2    |
|S     |6    |
|T     |2    |
|W     |1    |
+------+-----+



In [0]:
# %sql
# WITH date_series AS (
#   SELECT EXPLODE(SEQUENCE(0, 30)) AS day_offset
# )
# SELECT date_add('2012-10-01 00:00:00', day_offset) AS ts
# FROM date_series;

date_series = spark.range(0, 31).toDF("day_offset")
date_series.withColumn("ts", F.date_add(F.lit('2012-10-01').cast('date'), F.col("day_offset").cast('int'))) \
    .show(n=31, truncate=False)


+----------+----------+
|day_offset|ts        |
+----------+----------+
|0         |2012-10-01|
|1         |2012-10-02|
|2         |2012-10-03|
|3         |2012-10-04|
|4         |2012-10-05|
|5         |2012-10-06|
|6         |2012-10-07|
|7         |2012-10-08|
|8         |2012-10-09|
|9         |2012-10-10|
|10        |2012-10-11|
|11        |2012-10-12|
|12        |2012-10-13|
|13        |2012-10-14|
|14        |2012-10-15|
|15        |2012-10-16|
|16        |2012-10-17|
|17        |2012-10-18|
|18        |2012-10-19|
|19        |2012-10-20|
|20        |2012-10-21|
|21        |2012-10-22|
|22        |2012-10-23|
|23        |2012-10-24|
|24        |2012-10-25|
|25        |2012-10-26|
|26        |2012-10-27|
|27        |2012-10-28|
|28        |2012-10-29|
|29        |2012-10-30|
|30        |2012-10-31|
+----------+----------+



In [0]:
# %sql
# SELECT date_trunc('month', starttime) as month, count(*)
# FROM bookings
# GROUP BY month
# ORDER BY month

b = bookings_df.alias("b")

b.withColumn("month", F.trunc(F.col("b.starttime"), "MM")) \
  .groupBy("month") \
  .agg(F.count("*").alias("count")) \
  .orderBy("month") \
  .show()

+----------+-----+
|     month|count|
+----------+-----+
|2012-07-01|  658|
|2012-08-01| 1472|
|2012-09-01| 1913|
|2013-01-01|    1|
+----------+-----+

